## Python 数模模型
本教程来源于《FluentPython》第一章

Python最好的品质是一致性，当你带着其他面向对象语言的经验进入Python世界后，会对len(collection)的写法感到不适，当你进一步理解不适的原因和其背后庞大的设计思想，这事Python风格的关键，这种思想完全体现在Python数据模型上

### 从一摞Python风格的纸牌开始

In [5]:
import collections

Card = collections.namedtuple('Card',['rank','suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2,11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits 
                                        for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]
    

当我们想随机抽取一张卡牌， 可以调用`random.choice`方法

In [15]:
from random import choice
deck = FrenchDeck()
print(choice(deck))

Card(rank='7', suit='clubs')


现在你已经可以体会到特殊方法实现Python数据类型的两个好处
>* 作为你的类用户，他们不需要记住操作的各式名称
>* 可以更加方便的利用Python标准库，不用重新发明轮子

更有趣的是，因为你`__getitem__`将[]操作交给了self.cards列表，所以你的类自动支持切片操作！
下面是例子

In [21]:
# 抽取最上面的三张牌
print(deck[:3])
# 从第12张牌开始割13张抽牌 拿到4个A
print(deck[12::13])
# 另外，由于实现了了__getitem__方法，card方法可迭代
for card in deck:
    card

[Card(rank='2', suit='spades'), Card(rank='3', suit='spades'), Card(rank='4', suit='spades')]
[Card(rank='A', suit='spades'), Card(rank='A', suit='diamonds'), Card(rank='A', suit='clubs'), Card(rank='A', suit='hearts')]


##  数据结构

### 序列类型 

创造Python以前，Guido曾为ABC语言贡献代码。Python也从ABC语言继承了用统一的风格去处理序列数据这一点。不管是哪种数据结构，字符串、列表、字节序列、数组、XML元素、数据库查询结果。共用一套丰富的操作：迭代、切片、排序、拼接

深入理解Python中不同的序列类型，不但能让我们避免重新发明轮子，它们的API还能帮助我们把自己定义的API设计的跟原生序列一样，或者是跟未来可能出现的序列类型保持兼容

Python用C标准库实现了丰富的数据类型
> 容器序列
>* list 、tuple和collections.deque这些能存放不同类型的数据
> 扁平序列
>* str、bytes 、bytearray、memoryview、array.array、这类容器只能容纳一种类型

容器序列存放的是它们所包含的任意类型的对象引用，而扁平序列里存放的是值而不是引用。换句话说，扁平序列其实是一段连续的内存空间。由此可见扁平序列其实更加紧凑，但是它只能存放诸如字符、字节、数值这种基础类型

序列还可以分成可变序列和不可变序列
> 可变序列
>* lsit、bytearray、array.array、collections.deque、memoryview
> 不可变序列
>* tuple、str、bytes


### 列表推导

列表推导是构建列表的快捷方式，而生成器表达式则可以用来创建其他任何类型的序列

列表推导(<b>list comprehension</b>)
下面哪个表达更具可读性？

列表推导不会存在变量泄露问题、表达式内部变量和赋值只在局部起作用，表达式上下文同名变量还可以被正常引用，局部变量不会影响到它们

In [2]:
# Exp1 
symbols = '$%%^&'
codes = []
for symbol in symbols:
    codes.append(ord(symbol))
print(codes)

# Exp2
symbols = '$%%^&'
codes = [ord(symbol) for symbol in symbols]
print(codes)

[36, 37, 37, 94, 38]
[36, 37, 37, 94, 38]


列表推导笛卡尔积

In [3]:
colors = ['black', 'white']
sizes = ['S','M','L']
tshirts = [(color,size) for color in colors for size in sizes]
print(tshirts)

[('black', 'S'), ('black', 'M'), ('black', 'L'), ('white', 'S'), ('white', 'M'), ('white', 'L')]


### 生成器表达式
虽然可以使用列表推导初始化元祖、数组、其他类型。有时候生成器是更好的选择，因为其逐个产生元素

In [6]:
symbols = '%^&&)'
# 元祖
tuple(ord(symbol) for symbol in symbols)

(37, 94, 38, 38, 41)

### 元组
“不可变列表”并不能完全概括元组的特点，除了用作不可变列表，它还用于没有字段名的记录

In [17]:
coordinate = (33.9425,-118.408056)
lati, longi = coordinate # 元组拆包

# 可用*把课迭代对象拆成函数参数
t = (20,8)
print(divmod(*t))

# 利用拆包 获取文件名
import os
_, filename = os.path.split('/home/luciano/.ssh/idrsa.pub')

# 使用*获取剩余元素
a, b, *rest = range(5)

(2, 4)


## 字典和集合

字典这个数据结构活跃在所有Python程序的背后 标准库所有的映射类型都是利用dict来实现，它们有个共同的限制，只有可散列的数据类型才能用作这些映射里的键（只有键有要求，值不需要散列）
>什么是可散列的类型？
>如果一个对象是可散列的，那么在这个对象的生命周期中，它的散列值是不变的，而且这个对象需要实现__hash__方法,他还要有__qe__方法，这样才能跟其他键作比较。不可变数据类型都是可散列类型，frozenst也是可散列的；对于元组，只有一个元组包含的元素都是可散列的情况下，它才是散列的。

### 字典推导：将以键值对作为元素的对象构建出字典

In [5]:
DIAL_CODES = [(86,'China'),(91,'India'),(1,'United States')]
country_code = {country : code for code,country in DIAL_CODES}
COUNTRY_CODE = {country.upper() : code for code,country in DIAL_CODES}
COUNTRY_CODE

{'CHINA': 86, 'INDIA': 91, 'UNITED STATES': 1}

### 处理查找不到的键

In [15]:
import sys
import re

# 匹配字母、数字、下划线
WORD_RE = re.compile(r'\w+')

index = {}
with open(sys.argv[0],encoding='utf-8') as fp:
    for line_no, line in enumerate(fp):
        print(line)

"""Entry point for launching an IPython kernel.



This is separate from the ipykernel package so we can avoid doing imports until

after removing the cwd from sys.path.

"""



import sys



if __name__ == '__main__':

    # Remove the CWD from sys.path while we load stuff.

    # This is added back by InteractiveShellApp.init_path()

    if sys.path[0] == '':

        del sys.path[0]



    from ipykernel import kernelapp as app

    app.launch_new_instance()



In [10]:
import sys
with open(sys.argv[1],encoding='utf-8') as fp:
    

-f
